# Exp 39

### Single model to detect randomness

In [1]:
import os
import datetime
import tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict

import models
import report
import block_sampler
import batch_encoder
from dataset import Dataset
from report import Reporter
from trainer import Trainer
from batch_encoder import Dataset
from block_sampler import count_sectors, BlockSamplerByCategory, RandomSampler
from batch_encoder import xs_encoder_8bits_11, BatchEncoder
from filter_random import gen_rndchk_models, evaluate_rnd_model, filter_dataset

/home/atila.alr/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/atila.alr/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/atila.alr/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/atila.alr/.local/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [2]:
# raw_dataset_folder='/home/user/sample200'
raw_dataset_folder='../datasets/govdocs1/sample200'
random_dataset_folder='../datasets/random'
minimum=200
maximum=200
result_dir = 'results/exp39'

In [3]:
rawset = Dataset.new_from_folders(raw_dataset_folder).filter_min_max(minimum, maximum)
rawset = rawset.filter(lambda x: rawset.category_from(x) not in ['text', 'unk'])
rawset.rebuild_categories()
rndset = Dataset.new_from_folders(random_dataset_folder) #.filter_min_max(minimum, maximum)

In [64]:
import models
from block_sampler import BlockSamplerByCategory
from batch_encoder import BatchEncoder
from collections import namedtuple
import callbacks
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

TrainResults = namedtuple(
    'TrainResults', ['model', 'history', 'metrics', 'elapsed'])

def adjustSamplingProbs(model, tset, xs_encoder, target_sampler):
    datasets = tset.by_category()
    encoders = {}
    for cat in datasets:
        tsampler = BlockSamplerByCategory(tset)
        trsampler = RandomSampler(tsampler, ratio=0.0)
        tbenc = BatchEncoder(trsampler, 100, xs_encoder=xs_encoder)
        encoders[cat] = iter(tbenc)
    def f(epoch, logs):
        results = {}
        for cat in encoders:
            result = model.evaluate_generator(
                encoders[cat],
                steps=1
            )
            results[cat] = result[-1]
        cat_prob = {}
        for cat, x in results.items():
            cat_prob[cat] = 1 + int((1-x)*100)
        print(cat_prob)
        target_sampler.category_prob = cat_prob
    return f
    

class RandomTrainer:
    def __init__(self,
                 model,
                 xs_encoder='8bits_11',
                 validation_steps=20,
                 steps_per_epoch=10,
                 epochs=10000000,
                 max_seconds=10*60,
                 batch_size=400,
                 min_delta=1e-01,
                 patience=4):
        self.model = model
        self.xs_encoder = xs_encoder
        self.validation_steps = validation_steps
        self.steps_per_epoch = steps_per_epoch
        self.epochs = epochs
        self.max_seconds = max_seconds
        self.batch_size = batch_size
        self.min_delta = min_delta
        self.patience = patience

    def train(self, tset, vset):
        tsampler = BlockSamplerByCategory(tset)
        trsampler = RandomSampler(tsampler)
        tbenc = BatchEncoder(trsampler, self.batch_size,
                             xs_encoder=self.xs_encoder)

        vsampler = BlockSamplerByCategory(vset)
        vrsampler = RandomSampler(tsampler)
        vbenc = BatchEncoder(vrsampler, self.batch_size,
                             xs_encoder=self.xs_encoder)

        model = self.model

        timeIt = callbacks.TimeIt()

        history = model.fit_generator(iter(tbenc),
                                      validation_data=iter(vbenc),
                                      validation_steps=self.validation_steps,
                                      steps_per_epoch=self.steps_per_epoch,
                                      epochs=self.epochs,
                                      verbose=1,
                                      callbacks=[
            timeIt,
            # callbacks.SaveModel(os.path.join(result_dir, model.name + '.h5')),
            callbacks.TimeLimit(self.max_seconds),
            EarlyStopping(monitor='val_categorical_accuracy',
                          min_delta=self.min_delta, patience=self.patience),
            LambdaCallback(on_epoch_end=adjustSamplingProbs(model, tset,
                                                            self.xs_encoder, trsampler))
            # TensorBoard(
            #     log_dir=os.path.join(log_dir, model.name),
            #     # update_freq=3100,
            # ),
        ],
            use_multiprocessing=False,
            workers=0,
        )
        return TrainResults(
            model=model,
            history=history,
            metrics=['val_binary_accuracy', 'val_categorical_accuracy'],
            elapsed=timeIt.elapsed,
        )

In [65]:
model = models.C64_16_2pr_C32_4_2pr_C64_32_2pr_F_D(
        2, 8, 'softmax', 'categorical_crossentropy')

In [66]:
rt = RandomTrainer(model)

In [ ]:
result = rt.train(*rawset.rnd_split_fraction_by_category(0.5))

Epoch 1/10000000
10/10 [==============================] - 9s 930ms/step - loss: 0.5417 - categorical_accuracy: 0.7322 - binary_accuracy: 0.7322 - val_loss: 0.3872 - val_categorical_accuracy: 0.8520 - val_binary_accuracy: 0.8520
Epoch 2/10000000
10/10 [==============================] - 8s 784ms/step - loss: 0.3870 - categorical_accuracy: 0.8455 - binary_accuracy: 0.8455 - val_loss: 0.3660 - val_categorical_accuracy: 0.8379 - val_binary_accuracy: 0.8379
Epoch 3/10000000
10/10 [==============================] - 8s 796ms/step - loss: 0.3418 - categorical_accuracy: 0.8537 - binary_accuracy: 0.8537 - val_loss: 0.3292 - val_categorical_accuracy: 0.8609 - val_binary_accuracy: 0.8609
Epoch 4/10000000
10/10 [==============================] - 8s 779ms/step - loss: 0.3204 - categorical_accuracy: 0.8643 - binary_accuracy: 0.8643 - val_loss: 0.3037 - val_categorical_accuracy: 0.8885 - val_binary_accuracy: 0.8885
Epoch 5/10000000
10/10 [==============================] - 8s 791ms/step - loss: 0.2776 -

Epoch 14/10000000
10/10 [==============================] - 8s 780ms/step - loss: 0.2380 - categorical_accuracy: 0.9112 - binary_accuracy: 0.9112 - val_loss: 0.2351 - val_categorical_accuracy: 0.9130 - val_binary_accuracy: 0.9130
Epoch 15/10000000
10/10 [==============================] - 8s 788ms/step - loss: 0.2309 - categorical_accuracy: 0.9147 - binary_accuracy: 0.9147 - val_loss: 0.2488 - val_categorical_accuracy: 0.9094 - val_binary_accuracy: 0.9094
Epoch 16/10000000
10/10 [==============================] - 8s 781ms/step - loss: 0.2285 - categorical_accuracy: 0.9143 - binary_accuracy: 0.9143 - val_loss: 0.2369 - val_categorical_accuracy: 0.9109 - val_binary_accuracy: 0.9109
Epoch 17/10000000
10/10 [==============================] - 8s 781ms/step - loss: 0.2296 - categorical_accuracy: 0.9135 - binary_accuracy: 0.9135 - val_loss: 0.2344 - val_categorical_accuracy: 0.9114 - val_binary_accuracy: 0.9114
Epoch 18/10000000
10/10 [==============================] - 8s 772ms/step - loss: 0.2

Epoch 27/10000000
10/10 [==============================] - 8s 794ms/step - loss: 0.2222 - categorical_accuracy: 0.9175 - binary_accuracy: 0.9175 - val_loss: 0.2150 - val_categorical_accuracy: 0.9214 - val_binary_accuracy: 0.9214
Epoch 28/10000000
10/10 [==============================] - 8s 785ms/step - loss: 0.2173 - categorical_accuracy: 0.9215 - binary_accuracy: 0.9215 - val_loss: 0.2505 - val_categorical_accuracy: 0.9080 - val_binary_accuracy: 0.9080
Epoch 29/10000000
10/10 [==============================] - 7s 732ms/step - loss: 0.2225 - categorical_accuracy: 0.9178 - binary_accuracy: 0.9178 - val_loss: 0.2264 - val_categorical_accuracy: 0.9134 - val_binary_accuracy: 0.9134
Epoch 30/10000000
10/10 [==============================] - 7s 731ms/step - loss: 0.2284 - categorical_accuracy: 0.9137 - binary_accuracy: 0.9137 - val_loss: 0.2208 - val_categorical_accuracy: 0.9191 - val_binary_accuracy: 0.9191
Epoch 31/10000000
10/10 [==============================] - 7s 740ms/step - loss: 0.2

In [ ]:
plt.plot(result.history.history['val_categorical_accuracy'], 'o')

In [ ]:
result

In [ ]:
result.model.save()